In [1]:
!pip install crewai==0.30.11 crewai_tools==0.2.6 langchain_community==0.0.38

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [4]:
import os

# Define your API keys directly
openai_api_key = 'sk-xxxxxxxxxxxxxxxxxxxxxxxxxx'
serper_api_key = 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxx'

# # Set environment variables
os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = serper_api_key

In [5]:
# Providing current CV as an .md file
md_content = """
# Michael Ross

**Email**: michael.ross@specterlitt.com
**Phone**: +1 212 555 1234

## Professional Summary
Highly skilled Attorney with a unique blend of photographic memory and comprehensive legal knowledge. Known for quick thinking, unparalleled analytical abilities, and a deep commitment to justice. Expert in corporate law, litigation, and legal research. Proven track record of successfully handling complex cases and delivering favorable outcomes for high-profile clients.

## Professional Experience

### Pearson Specter Litt: Senior Associate Attorney — 2011 - Present
- Represented Fortune 500 companies in high-stakes corporate litigation, consistently achieving favorable settlements and verdicts.
- Developed and executed innovative legal strategies, leveraging a deep understanding of case law and precedents.
- Mentored junior associates, fostering a collaborative and high-performance culture within the firm.
- Spearheaded pro bono initiatives, providing legal assistance to underprivileged communities and enhancing the firm’s reputation.

### Freelance Legal Consultant — 2009 - 2011
- Provided legal consultancy services to various clients, specializing in contract law, intellectual property, and civil litigation.
- Successfully negotiated settlements in multiple complex cases, demonstrating exceptional negotiation skills and legal acumen.
- Conducted comprehensive legal research and analysis, delivering insightful and actionable advice to clients.

### Law Clerk: Private Practice — 2008 - 2009
- Assisted senior attorneys in preparing for trials, including drafting legal documents, conducting research, and organizing evidence.
- Gained hands-on experience in courtroom procedures, client consultations, and case management.
- Demonstrated a keen eye for detail and an ability to synthesize complex legal information quickly.

## Education
- **Juris Doctor (J.D.)**
  Harvard Law School (Uncompleted due to unique circumstances)

- **B.A. in History**
  University of Pennsylvania

## Skills
- **Photographic Memory**: Exceptional ability to recall detailed information and apply it to legal cases.
- **Analytical Thinking**: Strong capacity for analyzing complex legal issues and developing effective solutions.
- **Negotiation**: Proven track record of negotiating favorable outcomes in high-stakes situations.
- **Legal Research**: Expertise in conducting thorough legal research and leveraging findings to build strong cases.
- **Litigation**: Extensive experience in representing clients in court and achieving successful verdicts.

## Notable Cases
- **Gillies Industries vs. Pearson Hardman**: Played a pivotal role in winning a high-profile corporate litigation case, securing a multimillion-dollar settlement.
- **Liberty Railroads vs. Soloff**: Successfully defended Liberty Railroads in a class-action lawsuit, resulting in a landmark ruling in favor of the client.
- **People vs. Clifford Danner**: Provided pro bono legal representation, resulting in the exoneration of an innocent man wrongfully convicted of murder.

Michael Ross is a dedicated and resourceful attorney, renowned for his legal brilliance and commitment to justice. Despite unconventional pathways, his legal prowess and unwavering determination have cemented his place as a key player in the legal field.
"""

with open('Michael_Ross_Profile.md', 'w') as file:
    file.write(md_content)

In [6]:
# Import necessary tools from the crewai_tools package
from crewai_tools import (
  FileReadTool,        # Tool to read files
  ScrapeWebsiteTool,   # Tool to scrape content from websites
  MDXSearchTool,       # Tool to perform semantic searches on MDX files
  SerperDevTool        # Tool to perform search operations
)

# Initialize a search tool instance using SerperDevTool
search_tool = SerperDevTool()

# Initialize a website scraping tool instance using ScrapeWebsiteTool
scrape_tool = ScrapeWebsiteTool()

# Read the content of a Markdown file containing Michael_Ross' profile
read_resume = FileReadTool(file_path='./Michael_Ross_Profile.md')

# Perform a semantic search on the MDX file containing Michael_Ross' profile
semantic_search_old_resume = MDXSearchTool(mdx='./Michael_Ross_Profile.md')

## Creating Agents

In [7]:
# Agent 1: Legal Job Researcher
researcher = Agent(
    role="Legal Job Researcher",
    goal="Provide in-depth and insightful analysis "
         "of legal job postings to empower legal professionals.",
    tools=[scrape_tool, search_tool],
    verbose=True,
    backstory=(
        "As a Legal Job Researcher, your unparalleled expertise in analysing "
        "and extracting critical information from legal job postings sets you apart. "
        "You help legal professionals understand the essential qualifications and skills sought by employers, "
        "enabling them to craft highly targeted applications. Your insights are crucial for successful job searches, "
        "guiding candidates towards their ideal positions in the legal field."
    )
)

In [8]:
# Agent 2: Profiler
profiler = Agent(
    role="Personal Profiler for Engineers",
    goal="Do increditble research on job applicants "
         "to help them stand out in the job market",
    tools = [scrape_tool, search_tool,
             read_resume, semantic_search_old_resume],
    verbose=True,
    backstory=(
        "Equipped with analytical prowess, you dissect "
        "and synthesize information "
        "from diverse sources to craft comprehensive "
        "personal and professional profiles, laying the "
        "groundwork for personalized resume improvements."
    )
)

In [9]:
# Agent 3: Resume Strategist
resume_strategist = Agent(
    role="Resume Strategist for lawyers",
    goal="Enhance resumes to showcase unique strengths and stand out.",
    tools=[scrape_tool, search_tool, read_resume, semantic_search_old_resume],
    verbose=True,
    backstory=(
        "With a strategic mindset and meticulous attention to detail, "
        "you specialize in optimizing resumes to emphasize key skills "
        "and experiences, ensuring they align with the "
        "requirements of the job."
    )
)

In [10]:
# Agent 4: Interview Preparer
interview_preparer = Agent(
    role="Law Job Interview Preparer",
    goal="Develop tailored interview questions and talking points "
         "aligned with resumes and job requirements.",
    tools=[scrape_tool, search_tool, read_resume, semantic_search_old_resume],
    verbose=True,
    backstory=(
        "Your expertise lies in anticipating the nuances of interviews. "
        "You excel in crafting strategic questions and talking points "
        "that thoroughly prepare candidates for success. Your aim is to "
        "ensure they approach every aspect of the job application "
        "with confidence and clarity."
    )
)

## Creating Tasks

In [11]:
# Task for Researcher Agent: Extract Job Requirements
research_task = Task(
    description=(
        "Analyze the job posting URL provided ({job_posting_url}) "
        "to extract essential skills, experiences, and qualifications "
        "needed for the position. Utilize the specified tools to gather "
        "information and categorize the requirements accordingly."
    ),
    expected_output=(
        "A structured list detailing the job's requirements, "
        "including key skills, experiences and any qualifications needed."
    ),
    agent=researcher
    #async_execution=True
)

In [12]:
# Task for Profiler Agent: Compile Comprehensive Profile
profile_task = Task(
    description=(
        "Compile a detailed personal and professional profile "
        "using personal write-up and CV information provided"
        "({personal_writeup}). Utilize tools to extract and "
        "synthesize information from these sources."
    ),
    expected_output=(
        "A comprehensive profile document that includes skills, "
        "project experiences, contributions, interests, and "
        "communication style."
    ),
    agent=profiler
   # async_execution=True
)

- You can pass a list of tasks as `context` to a task.
- The task then takes into account the output of those tasks in its execution.
- The task will not run until it has the output(s) from those tasks.

In [13]:
# Task for Resume Strategist Agent: Align Resume with Job Requirements
resume_strategy_task = Task(
    description=(
        "Using the profile and job requirements gathered from "
        "previous tasks, customize the resume to emphasize the most "
        "relevant areas. Utilize tools to refine and enhance "
        "resume content. Ensure accuracy throughout all sections, "
        "including the summary, work experience, skills, and education, "
        "to best showcase the candidate's abilities and alignment "
        "with the job posting."
    ),
    expected_output=(
        "A refined resume with changes that effectively highlights the candidate's "
        "qualifications and experiences. Make sure to make some changes to the resume."
    ),
    output_file="tailored_resume.md",
    context=[research_task, profile_task],
    agent=resume_strategist
)

In [14]:
# Task for Interview Preparer Agent: Develop Interview Materials
interview_preparation_task = Task(
    description=(
        "Develop a set of interview questions and discussion points "
        "based on the tailored resume and job requirements. Utilize "
        "tools to generate relevant and insightful questions that "
        "help the candidate articulate key aspects of their experience "
        "and alignment with the job role."
    ),
    expected_output=(
        "A document containing essential interview questions and "
        "discussion points for the candidate to prepare for the initial "
        "interview."
    ),
    output_file="interview_materials.md",
    context=[research_task, profile_task, resume_strategy_task],
    agent=interview_preparer
)

## Creating the Crew

In [15]:
job_application_crew = Crew(
    agents=[researcher,
            profiler,
            resume_strategist,
            interview_preparer],

    tasks=[research_task,
           resume_strategy_task,
           interview_preparation_task],

    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [16]:
job_application_inputs = {
    'job_posting_url': 'https://jobs.thelawyer.com/job/1663111/corporate-senior-associatejunior-partner-manchester?LinkSource=SimilarJobPlatform',
    'personal_writeup': """Highly skilled Attorney with a unique blend of photographic memory and comprehensive legal knowledge.
    Known for quick thinking, unparalleled analytical abilities, and a deep commitment to justice.
    Expert in corporate law, litigation, and legal research. Proven track record of successfully handling complex cases and delivering
    favorable outcomes for high-profile clients."""
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
### this execution will take a few minutes to run
result = job_application_crew.kickoff(inputs=job_application_inputs)

 [DEBUG]: == Working Agent: Legal Job Researcher
 [INFO]: == Starting Task: Analyze the job posting URL provided (https://jobs.thelawyer.com/job/1663111/corporate-senior-associatejunior-partner-manchester?LinkSource=SimilarJobPlatform) to extract essential skills, experiences, and qualifications needed for the position. Utilize the specified tools to gather information and categorize the requirements accordingly.


> Entering new CrewAgentExecutor chain...
I need to carefully analyze the job posting URL provided to extract the essential skills, experiences, and qualifications needed for the position. I should use the tools available to gather information and categorize the requirements accordingly.

Action: Read website content
Action Input: {"website_url": "https://jobs.thelawyer.com/job/1663111/corporate-senior-associatejunior-partner-manchester?LinkSource=SimilarJobPlatform"} 

Corporate Senior Associate/Junior Partner - Manchester job with Think Legal Recruitment | 1663111
Skip to 

In [18]:
from IPython.display import Markdown, display
display(Markdown("./tailored_resume.md"))

# Michael Ross

**Email**: michael.ross@specterlitt.com
**Phone**: +1 212 555 1234

## Professional Summary
Highly skilled Attorney with a unique blend of photographic memory and comprehensive legal knowledge. Known for quick thinking, unparalleled analytical abilities, and a deep commitment to justice. Expert in corporate law, litigation, and legal research. Proven track record of successfully handling complex cases and delivering favorable outcomes for high-profile clients. Seeking a challenging Corporate Senior Associate/Junior Partner role in Manchester to leverage expertise in M&A, private equity, and transaction management.

## Professional Experience

### Pearson Specter Litt: Senior Associate Attorney — 2011 - Present
- Represented Fortune 500 companies in high-stakes corporate litigation, consistently achieving favorable settlements and verdicts.
- Developed and executed innovative legal strategies, leveraging a deep understanding of case law and precedents.
- Mentored junior associates, fostering a collaborative and high-performance culture within the firm.
- Spearheaded pro bono initiatives, providing legal assistance to underprivileged communities and enhancing the firm’s reputation.

### Freelance Legal Consultant — 2009 - 2011
- Provided legal consultancy services to various clients, specializing in contract law, intellectual property, and civil litigation.
- Successfully negotiated settlements in multiple complex cases, demonstrating exceptional negotiation skills and legal acumen.
- Conducted comprehensive legal research and analysis, delivering insightful and actionable advice to clients.

### Law Clerk: Private Practice — 2008 - 2009
- Assisted senior attorneys in preparing for trials, including drafting legal documents, conducting research, and organizing evidence.
- Gained hands-on experience in courtroom procedures, client consultations, and case management.
- Demonstrated a keen eye for detail and an ability to synthesize complex legal information quickly.

## Education
- **Juris Doctor (J.D.)**
  Harvard Law School (Uncompleted due to unique circumstances)

- **B.A. in History**
  University of Pennsylvania

## Skills
- **Photographic Memory**: Exceptional ability to recall detailed information and apply it to legal cases.
- **Analytical Thinking**: Strong capacity for analyzing complex legal issues and developing effective solutions.
- **Negotiation**: Proven track record of negotiating favorable outcomes in high-stakes situations.
- **Legal Research**: Expertise in conducting thorough legal research and leveraging findings to build strong cases.
- **Litigation**: Extensive experience in representing clients in court and achieving successful verdicts.

## Notable Cases
- **Gillies Industries vs. Pearson Hardman**: Played a pivotal role in winning a high-profile corporate litigation case, securing a multimillion-dollar settlement.
- **Liberty Railroads vs. Soloff**: Successfully defended Liberty Railroads in a class-action lawsuit, resulting in a landmark ruling in favor of the client.
- **People vs. Clifford Danner**: Provided pro bono legal representation, resulting in the exoneration of an innocent man wrongfully convicted of murder.

Michael Ross is a dedicated and resourceful attorney, renowned for his legal brilliance and commitment to justice. With a proven track record in high-stakes corporate litigation and a strong foundation in legal research, negotiation, and transaction management, he is well-equipped to excel in a Corporate Senior Associate/Junior Partner role in Manchester.

In [19]:
display(Markdown("./interview_materials.md"))

Based on Michael Ross's professional experience and skills outlined in his resume, as well as the job requirements for the Corporate Senior Associate/Junior Partner role in Manchester, here are tailored interview questions and discussion points for the candidate:

1. Can you provide examples of complex cases you have successfully handled in corporate litigation, particularly those involving Fortune 500 companies?
2. How have you demonstrated your ability to develop and execute innovative legal strategies in high-stakes situations? Can you walk us through a specific example?
3. In what ways have you mentored junior associates and fostered a collaborative culture within a legal team?
4. How have you used your negotiation skills to achieve favorable outcomes in legal cases, especially in complex and challenging scenarios?
5. Could you share a notable case where you provided pro bono legal assistance and the impact it had on the community or individuals involved?
6. Given your experience in legal research and analysis, how do you approach synthesizing complex legal information to build a strong case for your clients?
7. What key skills and qualities do you possess that make you well-equipped to lead on complex transactions and provide support to junior team members?
8. How do you balance handling big ticket corporate work with maintaining a healthy work/life balance, and how would you contribute to maintaining a positive work atmosphere in our firm?
9. Can you discuss a situation where you demonstrated your analytical thinking skills in resolving a complex legal issue effectively?
10. How would you leverage your expertise in M&A, private equity, and transaction management to contribute to the growth and success of our corporate team in Manchester?

These questions are designed to help the candidate showcase their experience, skills, and alignment with the job role, enabling them to articulate their qualifications effectively during the interview process.

In [20]:
from  openai import OpenAI
import os

# Set up OpenAI API key
client = OpenAI(api_key = openai_api_key)

def read_markdown_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

def get_differences(old_text, new_text):

    response = client.chat.completions.create(
        model='gpt-3.5-turbo-0125',
        messages=[
            {"role": "system", "content": "You are an assistant that helps compare documents."},
            {"role": "user", "content": (
        f"Compare the following two markdown texts and list the differences, be very thorough:\n\n"
        f"Old text:\n{old_text}\n\n"
        f"New text:\n{new_text}\n\n"
        f"Differences:"
    )}
        ],
        max_tokens=4000
    )

    #return response["choices"][0]["message"]["content"]
    return response.choices[0].message.content

# Read content from the markdown files
old_resume_path = './Michael_Ross_Profile.md'
new_resume_path = './tailored_resume.md'

old_resume_markdown = read_markdown_file(old_resume_path)
new_resume_markdown = read_markdown_file(new_resume_path)

# Get differences using OpenAI GPT
differences = get_differences(old_resume_markdown, new_resume_markdown)

# Print the differences
print("Differences:\n", differences)

Differences:
 Here are the differences between the old and new texts:

1. **Professional Summary**:
   - The new text adds a sentence at the end mentioning that Michael Ross is seeking a challenging Corporate Senior Associate/Junior Partner role in Manchester to leverage expertise in M&A, private equity, and transaction management.

2. **Education**:
   - Both texts have the same education section.

3. **Skills**:
   - There are no changes to the skills section.

4. **Notable Cases**:
   - The notable cases section is unchanged in the new text.

5. **Overall Content**:
   - The new text has an additional paragraph in the Professional Summary section highlighting Michael Ross's interest in pursuing a specific role in Manchester.
  
6. **Professional Experience**:
   - There are no changes in the Professional Experience section.

7. **Formatting**:
   - Both texts have the same formatting with headings, subheadings, and bullet points.

8. **Tone**:
   - The new text adds emphasis on Mich